## How to use
1. Click 'Run All' and the file opener will come up (make sure to check your background applications) 
2. Select the image containing the cells 
3. Image of the cells in a black background will display (check your background applications)
3. Click on any of the cells and the cell will be displayed
4. Click on the black background to either: 
    - Show the image back to how it looked before 
    - Show a different cell
        - (This is decided by if the clicked area has a cell or not)
5. To stop the image, press any key or close it manually 

## Code Explanation
#### 1st part
- Reading image using easygui
- Resizing image in case it is too large or small 
- Converting original image to grayscale for later use 

#### 2nd part
- Finding threshold using simple tresholding of the original image
- Masking the original image with the threshold - 32 (found it to be the best threshold for the images)
- Using mask and inverse mask to extract cell from image into a black background 

#### 3rd part 
- Finding contours and hierarchy of the inverse mask
- Using loop to enumerate the contours and check 
    - if there are any parent-child relationship only use the parent contour
    - if the contour area is higher than 100
- Drawing cotours in the ROI

#### 4th part 
- Function checking for user input (left click)
- Creating a completely black image to use for the selected cell [Reference][1]
- Using for loop to iterate on contours and uses pointPolygonTest function to check if the clicked area is inside a contour [Reference][2], [Reference2][3]
    - if inside the countour 
        - draws the contours
        - Merges the ROI and mask 
- Displays the cell

#### 5th part
- Creates a window for image 
- Calls draw function (4th part)
- Displays ROI and waits for user input (key)

[1]: https://datacarpentry.org/image-processing/04-drawing/ 
[2]: https://docs.opencv.org/3.4/d3/dc0/group__imgproc__shape.html#ga1a539e8db2135af2566103705d7a5722
[3]: https://docs.opencv.org/3.4/dc/d48/tutorial_point_polygon_test.html

In [1]:
# 1st part
import cv2
import easygui
import numpy as np

I = easygui.fileopenbox(filetypes=["*.jpg","*.jpeg","*.png"])
I = cv2.imread(I)

down_width = 1000
down_height = 500
down_points = (down_width, down_height)
I = cv2.resize(I, down_points, interpolation= cv2.INTER_LINEAR)

grayImage = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

In [2]:
# 2nd part
T = np.mean(I) + np.std(I)
T, Mask = cv2.threshold(I, thresh = T-32, maxval = 255, type = cv2.THRESH_BINARY)

gray_mask = cv2.cvtColor(Mask, cv2.COLOR_BGR2GRAY)
inverse_mask = cv2.bitwise_not(gray_mask)
ROI = cv2.bitwise_and(I, I, mask = inverse_mask)

In [3]:
# 3rd part
contours, hierarchy = cv2.findContours(inverse_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

for i,contour in enumerate(contours):
    if hierarchy[0][i][2] > 0 or hierarchy[0][i][2] < 0 :
        if  cv2.contourArea(contour) > 100:
            cv2.drawContours(ROI,[contour], -1, (0,255,0), 2)

In [4]:
# 4th part
def draw(event,x,y,flags,param): 
    h, w = I.shape[:2]
    mask = np.zeros((h,w), np.uint8)
    out = ROI
    if event == cv2.EVENT_LBUTTONDOWN: 
        for i in range(0, len(contours)):         
            clicked = cv2.pointPolygonTest(contours[i], (x, y), False)
        
            if clicked > 0:
                cv2.drawContours(mask, contours, i, 255, -1)
                out = cv2.bitwise_and(ROI, ROI, mask=mask)
        cv2.imshow("image", out)

In [5]:
# 5th part
cv2.namedWindow("image") 
cv2.setMouseCallback("image", draw) 
cv2.imshow("image", ROI)
key = cv2.waitKey(0)
cv2.destroyAllWindows()